In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Apr 16 16:49:26 2021

@author: pmarc
Résolution Problème L11 - #1 par Pierre-Marc Juneau, 17 avril 2021
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import metrics as mt
import statsmodels.api as sm
import scipy.stats as sts
import math
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [2]:

donnee = pd.read_csv('../Fichiers/Poissonsv0r2.csv')
stats=donnee.describe()
dimensions=donnee.shape
nomsvariables = pd.DataFrame(donnee.columns)

#X=donnee.drop('Espèces', 1)
X=pd.get_dummies(donnee)
X=X.drop('Espèces_Éperlan', 1)
MatriceR = X.corr()

X=X.drop('Poids (g)', 1)
Y=donnee["Poids (g)"]

donnee.head()


/tmp/ipykernel_212966/2148179218.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X=X.drop('Espèces_Éperlan', 1)
/tmp/ipykernel_212966/2148179218.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X=X.drop('Poids (g)', 1)


,Espèces,Poids (g),Longueur 1 (cm),Longueur 2 (cm),Longueur 3 (cm),Hauteur (cm),Épaisseur (cm)
0,Daurades,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Daurades,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Daurades,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Daurades,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Daurades,430.0,26.5,29.0,34.0,12.4440,5.1340


In [5]:

"Ronde 1"

VIF = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
VIFPanda=pd.DataFrame(VIF)
VIFPanda.index=X.columns
VIFPanda.columns=["VIF"]

modele=sm.OLS(Y,X.assign(const=1))
resultats=modele.fit()
Y_chap = resultats.predict(X.assign(const=1))
resultats.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Poids (g)   R-squared:                       0.936
Model:                            OLS   Adj. R-squared:                  0.931
Method:                 Least Squares   F-statistic:                     195.7
Date:                Thu, 16 Mar 2023   Prob (F-statistic):           6.85e-82
Time:                        18:59:35   Log-Likelihood:                -941.46
No. Observations:                 159   AIC:                             1907.
Df Residuals:                     147   BIC:                             1944.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Longueur 1 (cm)         -80.3030     36.279     -2.214      0.028    -151.998      -8.608
Longueur 2 (cm)          79.8886     45.718      1.747      0.083     -10.461     170.238
Longueur 3 (cm)          32.5354     29.300      1.110      0.269     -25.369      90.439
Hauteur (cm)              5.2510     13.056      0.402      0.688     -20.551      31.053
Épaisseur (cm)           -0.5154     23.913     -0.022      0.983     -47.773      46.742
Espèces_Brochet        -654.5027     74.067     -8.837      0.000    -800.877    -508.128
Espèces_Daurades       -446.0733    119.430     -3.735      0.000    -682.095    -210.051
Espèces_Gardon         -343.0334     47.145     -7.276      0.000    -436.204    -249.863
Espèces_Parkki         -281.3507     66.468     -4.233      0.000    -412.707    -149.995
Espèces_Perche         -308.1244     42.941     -7.176      0.000    -392.985    -223.263
Espèces_Poisson blanc  -352.1991     62.857     -5.603      0.000    -476.419    -227.979
const                  -472.2588     36.278    -13.018      0.000    -543.952    -400.566
==============================================================================
Omnibus:                       43.558   Durbin-Watson:                   0.973
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               97.422
Skew:                           1.184   Prob(JB):                     7.00e-22
Kurtosis:                       6.016   Cond. No.                     1.06e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.06e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:
print(VIFPanda)

                                VIF
Longueur 1 (cm)        18736.208535
Longueur 2 (cm)        32343.545401
Longueur 3 (cm)        13505.545252
Hauteur (cm)             286.898423
Épaisseur (cm)           217.915081
Espèces_Brochet            9.431636
Espèces_Daurades          56.461740
Espèces_Gardon             4.937299
Espèces_Parkki             5.118829
Espèces_Perche             8.649594
Espèces_Poisson blanc      2.629376


In [ ]:

"VIFs très élevés pour les 3 longueurs (même ordre de grandeur), et il y a des coefficients non-significatifs"
"Garder une seule longueur (elles sont très corrélées ensembles, donc capturent la même tendance)"
"Garder la longueur avec la plus petite p-value (par exemple)"


In [7]:

"Ronde 2"

X=X.drop({'Longueur 2 (cm)','Longueur 3 (cm)'}, 1)
VIF = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
VIFPanda=pd.DataFrame(VIF)
VIFPanda.index=X.columns
VIFPanda.columns=["VIF"]

modele=sm.OLS(Y,X.assign(const=1))
resultats=modele.fit()
Y_chap = resultats.predict(X.assign(const=1))
resultats.summary()

"VIFs élevés pour hauteur et épaisseur, avec des coefficients non-significatifs"
"Enlever ces variables"

"Ronde 3"

X=X.drop({'Hauteur (cm)','Épaisseur (cm)'}, 1)
VIF = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
VIFPanda=pd.DataFrame(VIF)
VIFPanda.index=X.columns
VIFPanda.columns=["VIF"]

modele=sm.OLS(Y,X.assign(const=1))
resultats=modele.fit()
Y_chap = resultats.predict(X.assign(const=1))
resultats.summary()


/tmp/ipykernel_212966/2210110072.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X=X.drop({'Longueur 2 (cm)','Longueur 3 (cm)'}, 1)
/tmp/ipykernel_212966/2210110072.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X=X.drop({'Hauteur (cm)','Épaisseur (cm)'}, 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Poids (g)   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.927
Method:                 Least Squares   F-statistic:                     286.9
Date:                Thu, 16 Mar 2023   Prob (F-statistic):           7.78e-84
Time:                        19:01:12   Log-Likelihood:                -948.61
No. Observations:                 159   AIC:                             1913.
Df Residuals:                     151   BIC:                             1938.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Longueur 1 (cm)          42.4320      1.221     34.741      0.000      40.019      44.845
Espèces_Brochet        -617.1721     51.723    -11.932      0.000    -719.366    -514.978
Espèces_Daurades       -201.6195     38.457     -5.243      0.000    -277.602    -125.637
Espèces_Gardon         -257.4744     35.638     -7.225      0.000    -327.887    -187.062
Espèces_Parkki         -173.3332     40.067     -4.326      0.000    -252.498     -94.168
Espèces_Perche         -243.2944     33.910     -7.175      0.000    -310.294    -176.295
Espèces_Poisson blanc  -224.5576     51.880     -4.328      0.000    -327.063    -122.053
const                  -466.4848     29.305    -15.918      0.000    -524.386    -408.584
==============================================================================
Omnibus:                       30.449   Durbin-Watson:                   0.862
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               55.264
Skew:                           0.910   Prob(JB):                     9.99e-13
Kurtosis:                       5.242   Cond. No.                         316.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:

"Tous les Betas sont significatifs, modèle et R2 intéressants"